In [58]:
'''
Notes:

Dense - A regular layer of neurons in a neural network.
Loss - It’s a method of evaluating how well your algorithm models your dataset. If your predictions are totally off, your loss function will output a higher number. If they’re pretty good, it’ll output a lower number.
Optimizer - Learning rate
Metrics - What parameters are being used to evaluate the overall quality of the ML


By category
'''

'\nNotes:\n\nDense - A regular layer of neurons in a neural network.\nLoss - It’s a method of evaluating how well your algorithm models your dataset. If your predictions are totally off, your loss function will output a higher number. If they’re pretty good, it’ll output a lower number.\nOptimizer - Learning rate\nMetrics - What parameters are being used to evaluate the overall quality of the ML\n\n'

In [59]:
import itertools
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import graphviz

from IPython.display import Image
from IPython.core.display import HTML
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
layers = keras.layers

print("Importing done")

Importing done


In [60]:
URL = "https://github.com/Brunel-Visualization/Brunel/raw/master/python/examples/data/whiskey.csv"
path = tf.keras.utils.get_file(URL.split('/')[-1], URL)

print("Data fetching done")

Data fetching done


In [61]:
# Convert the data to a Pandas data frame
data = pd.read_csv(path)

# Shuffle the data
data = data.sample(frac=1)

features = list(data.columns.values)
print(features)

print("Data reading done")

['Name', 'Rating', 'Country', 'Category', 'Price', 'ABV', 'Age', 'Brand']
Data reading done


In [62]:
# Do some preprocessing to limit the # of wine varities in the dataset
data = data[pd.notnull(data['Name'])]
data = data[pd.notnull(data['Category'])]
data = data[pd.notnull(data['Price'])]
data = data[pd.notnull(data['Rating'])]
data = data[pd.notnull(data['Age'])]

print("Data processing done")

Data processing done


In [63]:
# Split data into train and test
train_size = int(len(data) * .8)
print ("Total size: %d" % (len(data)))
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

print("Data splitting done")

Total size: 164
Train size: 131
Test size: 33
Data splitting done


In [64]:
# Train features
category_train = data['Age'][:train_size]
name_train = data['Name'][:train_size]

# Train labels
labels_trainOne = data['Price'][:train_size]
labels_trainTwo = data['Rating'][:train_size]

# Test features
category_test = data['Age'][:train_size]
name_test = data['Name'][:train_size]

# Test labels
labels_testOne = data['Price'][:train_size]
labels_testTwo = data['Rating'][:train_size]

print("Labeling done")

Labeling done


In [65]:
# Create a tokenizer to preprocess our text descriptions
vocab_size = 10 # This is a hyperparameter, experiment with different values for your dataset
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(name_train) # only fit on train

print("Tokeninzing done")

Tokeninzing done


In [66]:
# Wide feature 1: sparse bag of words (bow) vocab_size vector
name_bow_train = tokenize.texts_to_matrix(name_train)
name_bow_test = tokenize.texts_to_matrix(name_test)

In [67]:
# Wide feature 2: one-hot vector of variety categories

# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(category_train)
category_train = encoder.transform(category_train)
category_test = encoder.transform(category_test)
num_classes = np.max(category_train) + 1

# Convert labels to one hot
category_train = keras.utils.to_categorical(category_train, num_classes)
category_test = keras.utils.to_categorical(category_test, num_classes)

In [68]:
# Define our wide model with the functional API
bow_inputs = layers.Input(shape=(vocab_size,))
variety_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, variety_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, variety_inputs], outputs=predictions)
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
plot_model(wide_model, to_file='model_plotA.png', show_shapes=True, show_layer_names=True)

Image(url = "model_plotA.png", width = 500, height = 500)

In [69]:
# Deep model feature: word embeddings of wine descriptions
train_embed = tokenize.texts_to_sequences(name_train)
test_embed = tokenize.texts_to_sequences(name_test)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")

In [70]:
# Define our deep model with the Functional API
deep_inputs = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = keras.Model(inputs=deep_inputs, outputs=embed_out)
deep_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
plot_model(deep_model, to_file='model_plotB.png', show_shapes=True, show_layer_names=True)

Image(url = "model_plotB.png", width = 300, height = 300)

In [71]:
# Combine wide and deep into one model
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_modelOne = keras.Model(wide_model.input + [deep_model.input], merged_out)
combined_modelTwo = keras.Model(wide_model.input + [deep_model.input], merged_out)
plot_model(combined_modelOne, to_file='model_plotC.png', show_shapes=True, show_layer_names=True)
combined_modelOne.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
combined_modelTwo.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

Image(url = "model_plotC.png", width = 800, height = 800)

In [72]:
# Run training
combined_modelOne.fit([name_bow_train, category_train] + [train_embed], labels_trainOne, epochs=5000, batch_size=256)

Epoch 1/5000
131/131 [==============================] - 1s 8ms/step - loss: 19387.2734 - acc: 0.0000e+00
Epoch 2/5000
131/131 [==============================] - 0s 46us/step - loss: 19370.8379 - acc: 0.0000e+00
Epoch 3/5000
131/131 [==============================] - 0s 54us/step - loss: 19356.6621 - acc: 0.0000e+00
Epoch 4/5000
131/131 [==============================] - 0s 60us/step - loss: 19340.0098 - acc: 0.0000e+00
Epoch 5/5000
131/131 [==============================] - 0s 46us/step - loss: 19323.1172 - acc: 0.0000e+00
Epoch 6/5000
131/131 [==============================] - 0s 68us/step - loss: 19305.9590 - acc: 0.0000e+00
Epoch 7/5000
131/131 [==============================] - 0s 45us/step - loss: 19288.4863 - acc: 0.0000e+00
Epoch 8/5000
131/131 [==============================] - 0s 57us/step - loss: 19270.7383 - acc: 0.0000e+00
Epoch 9/5000
131/131 [==============================] - 0s 46us/step - loss: 19252.9160 - acc: 0.0000e+00
Epoch 10/5000
131/131 [========================

Epoch 78/5000
131/131 [==============================] - 0s 61us/step - loss: 15807.7490 - acc: 0.0000e+00
Epoch 79/5000
131/131 [==============================] - 0s 53us/step - loss: 15722.4277 - acc: 0.0000e+00
Epoch 80/5000
131/131 [==============================] - 0s 53us/step - loss: 15636.5586 - acc: 0.0000e+00
Epoch 81/5000
131/131 [==============================] - 0s 71us/step - loss: 15550.7109 - acc: 0.0000e+00
Epoch 82/5000
131/131 [==============================] - 0s 46us/step - loss: 15464.4287 - acc: 0.0000e+00
Epoch 83/5000
131/131 [==============================] - 0s 46us/step - loss: 15377.2393 - acc: 0.0000e+00
Epoch 84/5000
131/131 [==============================] - 0s 53us/step - loss: 15290.2100 - acc: 0.0000e+00
Epoch 85/5000
131/131 [==============================] - 0s 69us/step - loss: 15202.3867 - acc: 0.0000e+00
Epoch 86/5000
131/131 [==============================] - 0s 61us/step - loss: 15114.3213 - acc: 0.0000e+00
Epoch 87/5000
131/131 [==============

Epoch 155/5000
131/131 [==============================] - 0s 30us/step - loss: 10882.1533 - acc: 0.0000e+00
Epoch 156/5000
131/131 [==============================] - 0s 46us/step - loss: 10860.0107 - acc: 0.0000e+00
Epoch 157/5000
131/131 [==============================] - 0s 30us/step - loss: 10838.5693 - acc: 0.0000e+00
Epoch 158/5000
131/131 [==============================] - 0s 30us/step - loss: 10817.8555 - acc: 0.0000e+00
Epoch 159/5000
131/131 [==============================] - 0s 30us/step - loss: 10797.7695 - acc: 0.0000e+00
Epoch 160/5000
131/131 [==============================] - 0s 38us/step - loss: 10778.2715 - acc: 0.0000e+00
Epoch 161/5000
131/131 [==============================] - 0s 45us/step - loss: 10759.2803 - acc: 0.0000e+00
Epoch 162/5000
131/131 [==============================] - 0s 48us/step - loss: 10740.8193 - acc: 0.0000e+00
Epoch 163/5000
131/131 [==============================] - 0s 46us/step - loss: 10722.9082 - acc: 0.0000e+00
Epoch 164/5000
131/131 [====

Epoch 231/5000
131/131 [==============================] - 0s 45us/step - loss: 9862.5791 - acc: 0.0000e+00
Epoch 232/5000
131/131 [==============================] - 0s 31us/step - loss: 9850.9189 - acc: 0.0000e+00
Epoch 233/5000
131/131 [==============================] - 0s 38us/step - loss: 9839.2178 - acc: 0.0000e+00
Epoch 234/5000
131/131 [==============================] - 0s 34us/step - loss: 9827.4756 - acc: 0.0000e+00
Epoch 235/5000
131/131 [==============================] - 0s 45us/step - loss: 9815.7344 - acc: 0.0000e+00
Epoch 236/5000
131/131 [==============================] - 0s 38us/step - loss: 9803.9902 - acc: 0.0000e+00
Epoch 237/5000
131/131 [==============================] - 0s 35us/step - loss: 9792.2539 - acc: 0.0000e+00
Epoch 238/5000
131/131 [==============================] - 0s 46us/step - loss: 9780.4844 - acc: 0.0000e+00
Epoch 239/5000
131/131 [==============================] - 0s 46us/step - loss: 9768.6367 - acc: 0.0000e+00
Epoch 240/5000
131/131 [=============

Epoch 308/5000
131/131 [==============================] - 0s 46us/step - loss: 8874.2617 - acc: 0.0000e+00
Epoch 309/5000
131/131 [==============================] - 0s 61us/step - loss: 8860.0410 - acc: 0.0000e+00
Epoch 310/5000
131/131 [==============================] - 0s 53us/step - loss: 8845.7900 - acc: 0.0000e+00
Epoch 311/5000
131/131 [==============================] - 0s 38us/step - loss: 8831.5244 - acc: 0.0000e+00
Epoch 312/5000
131/131 [==============================] - 0s 30us/step - loss: 8817.2080 - acc: 0.0000e+00
Epoch 313/5000
131/131 [==============================] - 0s 38us/step - loss: 8802.8672 - acc: 0.0000e+00
Epoch 314/5000
131/131 [==============================] - 0s 46us/step - loss: 8788.5137 - acc: 0.0000e+00
Epoch 315/5000
131/131 [==============================] - 0s 38us/step - loss: 8774.0928 - acc: 0.0000e+00
Epoch 316/5000
131/131 [==============================] - 0s 53us/step - loss: 8759.6504 - acc: 0.0000e+00
Epoch 317/5000
131/131 [=============

...

Epoch 4689/5000
131/131 [==============================] - 0s 54us/step - loss: 375.3355 - acc: 0.0000e+00
Epoch 4690/5000
131/131 [==============================] - 0s 53us/step - loss: 375.3353 - acc: 0.0000e+00
Epoch 4691/5000
131/131 [==============================] - 0s 46us/step - loss: 375.3352 - acc: 0.0000e+00
Epoch 4692/5000
131/131 [==============================] - 0s 46us/step - loss: 375.3350 - acc: 0.0000e+00
Epoch 4693/5000
131/131 [==============================] - 0s 55us/step - loss: 375.3349 - acc: 0.0000e+00
Epoch 4694/5000
131/131 [==============================] - 0s 46us/step - loss: 375.3347 - acc: 0.0000e+00
Epoch 4695/5000
131/131 [==============================] - 0s 61us/step - loss: 375.3345 - acc: 0.0000e+00
Epoch 4696/5000
131/131 [==============================] - 0s 53us/step - loss: 375.3344 - acc: 0.0000e+00
Epoch 4697/5000
131/131 [==============================] - 0s 53us/step - loss: 375.3342 - acc: 0.0000e+00
Epoch 4698/5000
131/131 [============

Epoch 4766/5000
131/131 [==============================] - 0s 61us/step - loss: 375.3237 - acc: 0.0000e+00
Epoch 4767/5000
131/131 [==============================] - 0s 69us/step - loss: 375.3235 - acc: 0.0000e+00
Epoch 4768/5000
131/131 [==============================] - 0s 84us/step - loss: 375.3234 - acc: 0.0000e+00
Epoch 4769/5000
131/131 [==============================] - 0s 68us/step - loss: 375.3232 - acc: 0.0000e+00
Epoch 4770/5000
131/131 [==============================] - 0s 53us/step - loss: 375.3231 - acc: 0.0000e+00
Epoch 4771/5000
131/131 [==============================] - 0s 76us/step - loss: 375.3229 - acc: 0.0000e+00
Epoch 4772/5000
131/131 [==============================] - 0s 69us/step - loss: 375.3228 - acc: 0.0000e+00
Epoch 4773/5000
131/131 [==============================] - 0s 51us/step - loss: 375.3226 - acc: 0.0000e+00
Epoch 4774/5000
131/131 [==============================] - 0s 76us/step - loss: 375.3225 - acc: 0.0000e+00
Epoch 4775/5000
131/131 [============

Epoch 4843/5000
131/131 [==============================] - 0s 58us/step - loss: 375.3130 - acc: 0.0000e+00
Epoch 4844/5000
131/131 [==============================] - 0s 86us/step - loss: 375.3128 - acc: 0.0000e+00
Epoch 4845/5000
131/131 [==============================] - 0s 61us/step - loss: 375.3128 - acc: 0.0000e+00
Epoch 4846/5000
131/131 [==============================] - 0s 60us/step - loss: 375.3127 - acc: 0.0000e+00
Epoch 4847/5000
131/131 [==============================] - 0s 84us/step - loss: 375.3125 - acc: 0.0000e+00
Epoch 4848/5000
131/131 [==============================] - 0s 69us/step - loss: 375.3123 - acc: 0.0000e+00
Epoch 4849/5000
131/131 [==============================] - 0s 62us/step - loss: 375.3122 - acc: 0.0000e+00
Epoch 4850/5000
131/131 [==============================] - 0s 61us/step - loss: 375.3121 - acc: 0.0000e+00
Epoch 4851/5000
131/131 [==============================] - 0s 69us/step - loss: 375.3120 - acc: 0.0000e+00
Epoch 4852/5000
131/131 [============

Epoch 4920/5000
131/131 [==============================] - 0s 53us/step - loss: 375.3034 - acc: 0.0000e+00
Epoch 4921/5000
131/131 [==============================] - 0s 46us/step - loss: 375.3033 - acc: 0.0000e+00
Epoch 4922/5000
131/131 [==============================] - 0s 44us/step - loss: 375.3031 - acc: 0.0000e+00
Epoch 4923/5000
131/131 [==============================] - 0s 57us/step - loss: 375.3030 - acc: 0.0000e+00
Epoch 4924/5000
131/131 [==============================] - 0s 44us/step - loss: 375.3029 - acc: 0.0000e+00
Epoch 4925/5000
131/131 [==============================] - 0s 53us/step - loss: 375.3028 - acc: 0.0000e+00
Epoch 4926/5000
131/131 [==============================] - 0s 53us/step - loss: 375.3027 - acc: 0.0000e+00
Epoch 4927/5000
131/131 [==============================] - 0s 46us/step - loss: 375.3026 - acc: 0.0000e+00
Epoch 4928/5000
131/131 [==============================] - 0s 46us/step - loss: 375.3024 - acc: 0.0000e+00
Epoch 4929/5000
131/131 [============

Epoch 4997/5000
131/131 [==============================] - 0s 46us/step - loss: 375.2947 - acc: 0.0000e+00
Epoch 4998/5000
131/131 [==============================] - 0s 45us/step - loss: 375.2947 - acc: 0.0000e+00
Epoch 4999/5000
131/131 [==============================] - 0s 53us/step - loss: 375.2946 - acc: 0.0000e+00
Epoch 5000/5000
131/131 [==============================] - 0s 53us/step - loss: 375.2944 - acc: 0.0000e+00


In [73]:
combined_modelOne.evaluate([name_bow_test, category_test] + [test_embed], labels_testOne, batch_size=256)

131/131 [==============================] - 0s 2ms/step


[375.2943115234375, 0.0]

In [74]:
# Generate predictions
predictions = combined_modelOne.predict([name_bow_test, category_test] + [test_embed])

In [75]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 30
diff = 0
num_correct = 0
relative_tolarance = 0

for i in range(num_predictions):
    val = predictions[i]
    print(name_test.iloc[i])
    print('Predicted Price: ', val[0], 'Actual Price: ', labels_testOne.iloc[i], '\n')
    diff += abs(val[0] - labels_testOne.iloc[i])
    relative_tolarance = labels_testOne.iloc[i] * ((abs(val[0] - labels_testOne.iloc[i])) / labels_testOne.iloc[i])
    if(abs(val[0] - labels_testOne.iloc[i]) <= relative_tolarance):
        num_correct += 1

Elmer T Lee Bourbon Whiskey
Predicted Price:  29.999699 Actual Price:  30.0 

Basil Hayden's 8 Year Old Bourbon Whiskey
Predicted Price:  35.998966 Actual Price:  36.0 

Breckenridge Bourbon Whiskey
Predicted Price:  39.000286 Actual Price:  39.0 

Deanston 12 Year Old Single Malt Scotch
Predicted Price:  43.90076 Actual Price:  47.0 

The Singleton of Glendullan 12 Year Old Single Malt Scotch
Predicted Price:  43.90076 Actual Price:  35.0 

Old Parr 12 Year Old Blended Scotch
Predicted Price:  40.094746 Actual Price:  40.0 

Wild Turkey 81 Bourbon Whiskey
Predicted Price:  16.99445 Actual Price:  17.0 

Powers Gold Label Irish Whiskey
Predicted Price:  26.33414 Actual Price:  25.0 

Aberfeldy 12 Year Old Single Malt Scotch
Predicted Price:  43.90076 Actual Price:  30.0 

Aberlour 16 Year Old Single Malt Scotch
Predicted Price:  62.49445 Actual Price:  65.0 

Connemara 12 Year Old Peated Single Malt Irish Whiskey
Predicted Price:  67.49172 Actual Price:  90.0 

Glenfiddich 21 Year Old 

In [76]:
# Compare the average difference between actual price and the model's predicted price
print('Average prediction difference: ', diff / num_predictions)

# Print the accuracy
correct_percentage = (num_correct/num_predictions) * 100
print("Correct rate:", correct_percentage, "%")

Average prediction difference:  4.566351954142252
Correct rate: 80.0 %


In [77]:
# Run training
combined_modelTwo.fit([name_bow_train, category_train] + [train_embed], labels_trainTwo, epochs=5000, batch_size=256)

Epoch 1/5000
131/131 [==============================] - 1s 7ms/step - loss: 10580.4707 - acc: 0.0000e+00
Epoch 2/5000
131/131 [==============================] - 0s 46us/step - loss: 10424.2070 - acc: 0.0000e+00
Epoch 3/5000
131/131 [==============================] - 0s 52us/step - loss: 10278.2373 - acc: 0.0000e+00
Epoch 4/5000
131/131 [==============================] - 0s 46us/step - loss: 10151.7070 - acc: 0.0000e+00
Epoch 5/5000
131/131 [==============================] - 0s 46us/step - loss: 10023.8105 - acc: 0.0000e+00
Epoch 6/5000
131/131 [==============================] - 0s 55us/step - loss: 9894.8164 - acc: 0.0000e+00
Epoch 7/5000
131/131 [==============================] - 0s 46us/step - loss: 9765.4971 - acc: 0.0000e+00
Epoch 8/5000
131/131 [==============================] - 0s 46us/step - loss: 9634.0000 - acc: 0.0000e+00
Epoch 9/5000
131/131 [==============================] - 0s 46us/step - loss: 9506.7754 - acc: 0.0000e+00
Epoch 10/5000
131/131 [============================

Epoch 79/5000
131/131 [==============================] - 0s 49us/step - loss: 4660.1431 - acc: 0.0000e+00
Epoch 80/5000
131/131 [==============================] - 0s 53us/step - loss: 4623.7578 - acc: 0.0000e+00
Epoch 81/5000
131/131 [==============================] - 0s 46us/step - loss: 4587.8193 - acc: 0.0000e+00
Epoch 82/5000
131/131 [==============================] - 0s 53us/step - loss: 4552.3208 - acc: 0.0000e+00
Epoch 83/5000
131/131 [==============================] - 0s 53us/step - loss: 4517.2539 - acc: 0.0000e+00
Epoch 84/5000
131/131 [==============================] - 0s 53us/step - loss: 4482.5786 - acc: 0.0000e+00
Epoch 85/5000
131/131 [==============================] - 0s 55us/step - loss: 4448.3203 - acc: 0.0000e+00
Epoch 86/5000
131/131 [==============================] - 0s 46us/step - loss: 4414.6279 - acc: 0.0000e+00
Epoch 87/5000
131/131 [==============================] - 0s 46us/step - loss: 4381.1411 - acc: 0.0000e+00
Epoch 88/5000
131/131 [=======================

Epoch 156/5000
131/131 [==============================] - 0s 53us/step - loss: 2771.8738 - acc: 0.0000e+00
Epoch 157/5000
131/131 [==============================] - 0s 45us/step - loss: 2755.5740 - acc: 0.0000e+00
Epoch 158/5000
131/131 [==============================] - 0s 52us/step - loss: 2739.3623 - acc: 0.0000e+00
Epoch 159/5000
131/131 [==============================] - 0s 53us/step - loss: 2723.2871 - acc: 0.0000e+00
Epoch 160/5000
131/131 [==============================] - 0s 46us/step - loss: 2707.3376 - acc: 0.0000e+00
Epoch 161/5000
131/131 [==============================] - 0s 53us/step - loss: 2691.5142 - acc: 0.0000e+00
Epoch 162/5000
131/131 [==============================] - 0s 53us/step - loss: 2675.8589 - acc: 0.0000e+00
Epoch 163/5000
131/131 [==============================] - 0s 46us/step - loss: 2660.2725 - acc: 0.0000e+00
Epoch 164/5000
131/131 [==============================] - 0s 53us/step - loss: 2644.8269 - acc: 0.0000e+00
Epoch 165/5000
131/131 [=============

Epoch 233/5000
131/131 [==============================] - 0s 47us/step - loss: 1826.4115 - acc: 0.0000e+00
Epoch 234/5000
131/131 [==============================] - 0s 56us/step - loss: 1817.4159 - acc: 0.0000e+00
Epoch 235/5000
131/131 [==============================] - 0s 53us/step - loss: 1808.4734 - acc: 0.0000e+00
Epoch 236/5000
131/131 [==============================] - 0s 53us/step - loss: 1799.5867 - acc: 0.0000e+00
Epoch 237/5000
131/131 [==============================] - 0s 53us/step - loss: 1790.7529 - acc: 0.0000e+00
Epoch 238/5000
131/131 [==============================] - 0s 46us/step - loss: 1781.9684 - acc: 0.0000e+00
Epoch 239/5000
131/131 [==============================] - 0s 53us/step - loss: 1773.2394 - acc: 0.0000e+00
Epoch 240/5000
131/131 [==============================] - 0s 54us/step - loss: 1764.6075 - acc: 0.0000e+00
Epoch 241/5000
131/131 [==============================] - 0s 61us/step - loss: 1756.0153 - acc: 0.0000e+00
Epoch 242/5000
131/131 [=============

Epoch 310/5000
131/131 [==============================] - 0s 53us/step - loss: 1282.3032 - acc: 0.0000e+00
Epoch 311/5000
131/131 [==============================] - 0s 54us/step - loss: 1276.7545 - acc: 0.0000e+00
Epoch 312/5000
131/131 [==============================] - 0s 53us/step - loss: 1271.2472 - acc: 0.0000e+00
Epoch 313/5000
131/131 [==============================] - 0s 53us/step - loss: 1265.7729 - acc: 0.0000e+00
Epoch 314/5000
131/131 [==============================] - 0s 48us/step - loss: 1260.3141 - acc: 0.0000e+00
Epoch 315/5000
131/131 [==============================] - 0s 52us/step - loss: 1254.8933 - acc: 0.0000e+00
Epoch 316/5000
131/131 [==============================] - 0s 46us/step - loss: 1249.5299 - acc: 0.0000e+00
Epoch 317/5000
131/131 [==============================] - 0s 46us/step - loss: 1244.2117 - acc: 0.0000e+00
Epoch 318/5000
131/131 [==============================] - 0s 53us/step - loss: 1238.9159 - acc: 0.0000e+00
Epoch 319/5000
131/131 [=============

...

Epoch 4675/5000
131/131 [==============================] - 0s 61us/step - loss: 18.7293 - acc: 0.0000e+00
Epoch 4676/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7292 - acc: 0.0000e+00
Epoch 4677/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7291 - acc: 0.0000e+00
Epoch 4678/5000
131/131 [==============================] - 0s 35us/step - loss: 18.7290 - acc: 0.0000e+00
Epoch 4679/5000
131/131 [==============================] - 0s 38us/step - loss: 18.7289 - acc: 0.0000e+00
Epoch 4680/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7288 - acc: 0.0000e+00
Epoch 4681/5000
131/131 [==============================] - 0s 60us/step - loss: 18.7287 - acc: 0.0000e+00
Epoch 4682/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7286 - acc: 0.0000e+00
Epoch 4683/5000
131/131 [==============================] - 0s 53us/step - loss: 18.7286 - acc: 0.0000e+00
Epoch 4684/5000
131/131 [=====================

Epoch 4753/5000
131/131 [==============================] - 0s 26us/step - loss: 18.7225 - acc: 0.0000e+00
Epoch 4754/5000
131/131 [==============================] - 0s 38us/step - loss: 18.7224 - acc: 0.0000e+00
Epoch 4755/5000
131/131 [==============================] - 0s 34us/step - loss: 18.7223 - acc: 0.0000e+00
Epoch 4756/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7222 - acc: 0.0000e+00
Epoch 4757/5000
131/131 [==============================] - 0s 61us/step - loss: 18.7221 - acc: 0.0000e+00
Epoch 4758/5000
131/131 [==============================] - 0s 53us/step - loss: 18.7221 - acc: 0.0000e+00
Epoch 4759/5000
131/131 [==============================] - 0s 54us/step - loss: 18.7220 - acc: 0.0000e+00
Epoch 4760/5000
131/131 [==============================] - 0s 69us/step - loss: 18.7219 - acc: 0.0000e+00
Epoch 4761/5000
131/131 [==============================] - 0s 61us/step - loss: 18.7218 - acc: 0.0000e+00
Epoch 4762/5000
131/131 [=====================

Epoch 4831/5000
131/131 [==============================] - 0s 41us/step - loss: 18.7162 - acc: 0.0000e+00
Epoch 4832/5000
131/131 [==============================] - 0s 39us/step - loss: 18.7161 - acc: 0.0000e+00
Epoch 4833/5000
131/131 [==============================] - 0s 38us/step - loss: 18.7161 - acc: 0.0000e+00
Epoch 4834/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7160 - acc: 0.0000e+00
Epoch 4835/5000
131/131 [==============================] - 0s 54us/step - loss: 18.7159 - acc: 0.0000e+00
Epoch 4836/5000
131/131 [==============================] - 0s 61us/step - loss: 18.7158 - acc: 0.0000e+00
Epoch 4837/5000
131/131 [==============================] - 0s 53us/step - loss: 18.7157 - acc: 0.0000e+00
Epoch 4838/5000
131/131 [==============================] - 0s 61us/step - loss: 18.7157 - acc: 0.0000e+00
Epoch 4839/5000
131/131 [==============================] - 0s 69us/step - loss: 18.7156 - acc: 0.0000e+00
Epoch 4840/5000
131/131 [=====================

Epoch 4909/5000
131/131 [==============================] - 0s 38us/step - loss: 18.7105 - acc: 0.0000e+00
Epoch 4910/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7104 - acc: 0.0000e+00
Epoch 4911/5000
131/131 [==============================] - 0s 38us/step - loss: 18.7104 - acc: 0.0000e+00
Epoch 4912/5000
131/131 [==============================] - 0s 53us/step - loss: 18.7104 - acc: 0.0000e+00
Epoch 4913/5000
131/131 [==============================] - 0s 52us/step - loss: 18.7103 - acc: 0.0000e+00
Epoch 4914/5000
131/131 [==============================] - 0s 60us/step - loss: 18.7103 - acc: 0.0000e+00
Epoch 4915/5000
131/131 [==============================] - 0s 64us/step - loss: 18.7104 - acc: 0.0000e+00
Epoch 4916/5000
131/131 [==============================] - 0s 68us/step - loss: 18.7104 - acc: 0.0000e+00
Epoch 4917/5000
131/131 [==============================] - 0s 63us/step - loss: 18.7105 - acc: 0.0000e+00
Epoch 4918/5000
131/131 [=====================

Epoch 4987/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7051 - acc: 0.0000e+00
Epoch 4988/5000
131/131 [==============================] - 0s 38us/step - loss: 18.7050 - acc: 0.0000e+00
Epoch 4989/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7049 - acc: 0.0000e+00
Epoch 4990/5000
131/131 [==============================] - 0s 31us/step - loss: 18.7048 - acc: 0.0000e+00
Epoch 4991/5000
131/131 [==============================] - 0s 38us/step - loss: 18.7048 - acc: 0.0000e+00
Epoch 4992/5000
131/131 [==============================] - 0s 38us/step - loss: 18.7047 - acc: 0.0000e+00
Epoch 4993/5000
131/131 [==============================] - 0s 31us/step - loss: 18.7046 - acc: 0.0000e+00
Epoch 4994/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7046 - acc: 0.0000e+00
Epoch 4995/5000
131/131 [==============================] - 0s 46us/step - loss: 18.7045 - acc: 0.0000e+00
Epoch 4996/5000
131/131 [=====================

In [78]:
combined_modelTwo.evaluate([name_bow_test, category_test] + [test_embed], labels_testTwo, batch_size=256)

131/131 [==============================] - 0s 2ms/step


[18.70410919189453, 0.0]

In [79]:
# Generate predictions
predictions = combined_modelTwo.predict([name_bow_test, category_test] + [test_embed])

In [80]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 30
diff = 0
num_correct = 0

for i in range(num_predictions):
    val = predictions[i]
    print(name_test.iloc[i])
    print('Predicted Rating: ', val[0], 'Actual Rating: ', labels_testTwo.iloc[i], '\n')
    diff += abs(val[0] - labels_testTwo.iloc[i])
    if(abs(val[0] - labels_testTwo.iloc[i]) <= 10):
        num_correct += 1

Elmer T Lee Bourbon Whiskey
Predicted Rating:  87.0037 Actual Rating:  87.0 

Basil Hayden's 8 Year Old Bourbon Whiskey
Predicted Rating:  80.00149 Actual Rating:  80.0 

Breckenridge Bourbon Whiskey
Predicted Rating:  88.00357 Actual Rating:  88.0 

Deanston 12 Year Old Single Malt Scotch
Predicted Rating:  93.10043 Actual Rating:  92.0 

The Singleton of Glendullan 12 Year Old Single Malt Scotch
Predicted Rating:  93.10043 Actual Rating:  89.0 

Old Parr 12 Year Old Blended Scotch
Predicted Rating:  86.268585 Actual Rating:  86.0 

Wild Turkey 81 Bourbon Whiskey
Predicted Rating:  80.99872 Actual Rating:  81.0 

Powers Gold Label Irish Whiskey
Predicted Rating:  80.00114 Actual Rating:  84.0 

Aberfeldy 12 Year Old Single Malt Scotch
Predicted Rating:  93.10043 Actual Rating:  96.0 

Aberlour 16 Year Old Single Malt Scotch
Predicted Rating:  91.50506 Actual Rating:  89.0 

Connemara 12 Year Old Peated Single Malt Irish Whiskey
Predicted Rating:  81.499794 Actual Rating:  73.0 

Glenf

In [81]:
# Compare the average difference between actual rating and the model's predicted rating
print('Average prediction difference: ', diff / num_predictions)

# Print the accuracy
correct_percentage = (num_correct/num_predictions) * 100
print("Correct rate:", correct_percentage, "%")

Average prediction difference:  2.0058609008789063
Correct rate: 100.0 %
